In [1]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
import torch

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from detectron2.data.datasets import register_coco_instances

# Evaluate on validation set

In [2]:
dataset = 'coco2017'
annot_dir = './coco2017/annotations'
imgs_dir = './coco2017/{}2017'

#annot_dir = './coco_test_annotations'
#imgs_dir = './coco_test2017'

for split in ['train', 'val']:
    annot_path = os.path.join(annot_dir, f'instances_{split}2017.json')
    d_name = dataset + f'_{split}'
    register_coco_instances(d_name, {}, annot_path, imgs_dir.format(split))


#annot_path = os.path.join(annot_dir, 'image_info_test-dev2017.json')
#register_coco_instances('coco2017_test-dev', {}, annot_path, imgs_dir)

# Load dataset
dataset_dicts = DatasetCatalog.get('coco2017_val')
metadata = MetadataCatalog.get('coco2017_val')

[05/01 16:47:14 d2.data.datasets.coco]: Loaded 5000 images in COCO format from ./coco2017/annotations/instances_val2017.json


In [3]:
from detectron2.model_zoo import get_config
from detectron2.config import LazyConfig
from detectron2.config.instantiate import instantiate
from detectron2.engine import DefaultPredictor

cfg = LazyConfig.load("khang_net/configs/huflit_net/huflit_net_se_1x.py")

cfg.train.device = 'cpu'
cfg.dataloader.evaluator.dataset_name = 'coco2017_val'
cfg.dataloader.train.dataset.names = 'coco2017_val'
cfg.dataloader.test.dataset.names = 'coco2017_val'
cfg.dataloader.train.total_batch_size = 16

cfg.model.num_classes = 80
cfg.model.yolof.num_classes = 80
cfg.model.mask_head.num_classes = 80
#cfg.model.yolof.score_thresh_test = 0.7
#cfg.model.yolof.max_detections_per_image = 50

cfg.train.init_checkpoint = "/Users/giakhang/Downloads/coco_huflit_net_ckpts/huflit_net_210k_iters/model_best.pth"

In [5]:
model = instantiate(cfg.model)

In [6]:
params = model.parameters()

In [7]:
total_params = sum(p.numel() for p in params)
print("Total parameters:", total_params)

Total parameters: 49846812


In [6]:
from detectron2.evaluation import inference_on_dataset

inference_on_dataset(
            model,
            instantiate(cfg.dataloader.test),
            instantiate(cfg.dataloader.evaluator),
)

[05/01 10:50:18 d2.data.datasets.coco]: Loaded 5000 images in COCO format from ./coco2017/annotations/instances_val2017.json
[05/01 10:50:18 d2.data.build]: Distribution of instances among all 80 categories:
|   category    | #instances   |   category   | #instances   |   category    | #instances   |
|:-------------:|:-------------|:------------:|:-------------|:-------------:|:-------------|
|    person     | 10777        |   bicycle    | 314          |      car      | 1918         |
|  motorcycle   | 367          |   airplane   | 143          |      bus      | 283          |
|     train     | 190          |    truck     | 414          |     boat      | 424          |
| traffic light | 634          | fire hydrant | 101          |   stop sign   | 75           |
| parking meter | 60           |    bench     | 411          |     bird      | 427          |
|      cat      | 202          |     dog      | 218          |     horse     | 272          |
|     sheep     | 354          |     cow

/Users/giakhang/miniconda3/envs/hope_to_public_net/lib/python3.9/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1708025539291/work/aten/src/ATen/native/TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[05/01 10:50:35 d2.evaluation.evaluator]: Inference done 11/5000. Dataloading: 0.0004 s/iter. Inference: 1.3776 s/iter. Eval: 0.0305 s/iter. Total: 1.4084 s/iter. ETA=1:57:06
[05/01 10:50:41 d2.evaluation.evaluator]: Inference done 15/5000. Dataloading: 0.0004 s/iter. Inference: 1.3459 s/iter. Eval: 0.0282 s/iter. Total: 1.3747 s/iter. ETA=1:54:12
[05/01 10:50:47 d2.evaluation.evaluator]: Inference done 20/5000. Dataloading: 0.0004 s/iter. Inference: 1.2786 s/iter. Eval: 0.0279 s/iter. Total: 1.3071 s/iter. ETA=1:48:29
[05/01 10:50:53 d2.evaluation.evaluator]: Inference done 24/5000. Dataloading: 0.0004 s/iter. Inference: 1.3191 s/iter. Eval: 0.0280 s/iter. Total: 1.3478 s/iter. ETA=1:51:46
[05/01 10:50:58 d2.evaluation.evaluator]: Inference done 28/5000. Dataloading: 0.0004 s/iter. Inference: 1.3120 s/iter. Eval: 0.0277 s/iter. Total: 1.3403 s/iter. ETA=1:51:03
[05/01 10:51:03 d2.evaluation.evaluator]: Inference done 31/5000. Dataloading: 0.0005 s/iter. Inference: 1.3624 s/iter. Eval:

OrderedDict([('bbox',
              {'AP': 0.000290592770739725,
               'AP50': 0.0006831314736428006,
               'AP75': 0.0001506667699674469,
               'APs': 0.0,
               'APm': 0.0,
               'APl': 0.0011636449867971673,
               'AP-person': 0.0,
               'AP-bicycle': 0.0,
               'AP-car': 0.0,
               'AP-motorcycle': 0.0075377749862556385,
               'AP-airplane': 0.0013302428742140518,
               'AP-bus': 0.0,
               'AP-train': 0.0,
               'AP-truck': 0.0,
               'AP-boat': 0.0003062972079701339,
               'AP-traffic light': 0.0,
               'AP-fire hydrant': 0.0,
               'AP-stop sign': 0.0,
               'AP-parking meter': 0.0,
               'AP-bench': 0.0,
               'AP-bird': 0.0,
               'AP-cat': 0.0,
               'AP-dog': 0.0,
               'AP-horse': 0.0,
               'AP-sheep': 0.0008011590100345167,
               'AP-cow': 0.000506013